<style>
    h1 {
    font-family: 'Segoe UI Semibold';
    font-size: 35px;
    text-align: center;
  }
</style>

# Taller 9: Bases de Datos en la Nube (RDS)


<style>
  .image {
            overflow: hidden;
            width: 100%;
            border-radius: 20px;
            object-fit: cover;
            object-position: top center;
        }
  .container {
            width: 100%;
            height: 350px;
        }

</style>
<div class="container">
    <img src= "./img/sql-cloud.png" class="image"/>
</div>

<style>
  p {
    font-family: 'Segoe UI Light';
    font-size: 20px;
    text-align: center;
  }
</style>

<div style="text-align: center;">
  Analítica Computacional para la Toma de Decisiones IIND-4130<br>
  Universidad de los Andes &copy; 2024 <br>
  21 de octubre de 2024 <br><br>
  <strong>GRUPO:</strong> <br>
  Camilo Duque - 202024289<br>
  Sofia Buitrago – 202020742<br>
</div>


### **Importar Librerias**

In [51]:
import psycopg2
import pandas as pd

### **Conectarse a Base de Datos (bd_postgres)**

In [52]:
engine = psycopg2.connect(
dbname="bd_postgres",
user="cduquec",
password="1234Camilo",
host="database-camilo.c1y0em2ay3hc.us-east-1.rds.amazonaws.com",
port='5432'
)

### **Conectarse a Base de Datos (World)**

In [53]:
engine = psycopg2.connect(
dbname="world",
user="cduquec",
password="1234Camilo",
host="database-camilo.c1y0em2ay3hc.us-east-1.rds.amazonaws.com",
port='5432'
)

cursor = engine.cursor()

#### Consulta - Nombre de Tablas de la BD

In [131]:
query = """
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public';
    """
cursor.execute(query)
tables = cursor.fetchall()

# Imprimir Tablas
print("Tablas en la base de datos:")
df = pd.DataFrame(tables, columns=[cursor.description[0][0]])
df


Tablas en la base de datos:


,table_name
0,city
1,country
2,country_language


#### Consulta 1 - Estructura de tablas BD

In [127]:
query = """
    SELECT table_name, column_name, data_type
    FROM information_schema.columns
    WHERE table_schema = 'public'
    ORDER BY table_name, ordinal_position; 
    """
cursor.execute(query)
result = cursor.fetchall()
df = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])
df

,table_name,column_name,data_type
0,city,id,integer
1,city,name,text
2,city,country_code,character
3,city,district,text
4,city,population,integer
5,country,code,character
6,country,name,text
7,country,continent,text
8,country,region,text
9,country,surface_area,real


#### Consulta 2 - Top 5 paises con más alto GNP per capita

In [138]:
try:
  query = """
      SELECT *, gnp/population AS gnp_per_capita
      FROM country
      WHERE population != 0
      ORDER BY gnp_per_capita DESC
      LIMIT 5; 
      """
  cursor.execute(query)
  result = cursor.fetchall()
  df = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])
except Exception as e:
    print(f"An error occurred: {e}")
    # Rollback to clear the transaction state
    cursor.connection.rollback()
df

,code,name,continent,region,surface_area,indep_year,population,life_expectancy,gnp,gnp_old,local_name,government_form,head_of_state,capital,code2,gnp_per_capita
0,LUX,Luxembourg,Europe,Western Europe,2586.0,1867.0,435700,77.1,16321.00,15519.00,Luxembourg/Lëtzebuerg,Constitutional Monarchy,Henri,2452,LU,0.03745926095937571724
1,CHE,Switzerland,Europe,Western Europe,41284.0,1499.0,7160400,79.6,264478.00,256092.00,Schweiz/Suisse/Svizzera/Svizra,Federation,Adolf Ogi,3248,CH,0.03693620468130272052
2,BMU,Bermuda,North America,North America,53.0,NaN,65000,76.9,2328.00,2190.00,Bermuda,Dependent Territory of the UK,Elisabeth II,191,BM,0.03581538461538461538
3,BRN,Brunei,Asia,Southeast Asia,5765.0,1984.0,328000,73.6,11705.00,12460.00,Brunei Darussalam,Monarchy (Sultanate),Haji Hassan al-Bolkiah,538,BN,0.03568597560975609756
4,LIE,Liechtenstein,Europe,Western Europe,160.0,1806.0,32300,78.8,1119.00,1084.00,Liechtenstein,Constitutional Monarchy,Hans-Adam II,2446,LI,0.03464396284829721362


#### Consulta 3 - Top 10 Ciudades de Sur America con más alto porcentaje de población con respecto al país

In [205]:
try:
  query = """
      SELECT city.name AS nombre_ciudad, country.name AS nombre_pais, city.population AS poblacion_ciudad, country.population AS poblacion_pais, (CAST(city.population AS FLOAT)/CAST(country.population AS FLOAT))*100 AS porc_pob
      FROM city
      LEFT JOIN country ON city.country_code = country.code
      WHERE continent = 'South America'
      ORDER BY porc_pOb DESC
      LIMIT 10; 
      """
  cursor.execute(query)
  result = cursor.fetchall()
  df = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])
except Exception as e:
    print(f"An error occurred: {e}")
    # Rollback to clear the transaction state
    cursor.connection.rollback()
df

,nombre_ciudad,nombre_pais,poblacion_ciudad,poblacion_pais,porc_pob
0,Stanley,Falkland Islands,1636,2000,81.800000
1,Montevideo,Uruguay,1236000,3337000,37.039257
2,Santiago de Chile,Chile,4703954,15211000,30.924686
3,Georgetown,Guyana,254000,861000,29.500581
4,Cayenne,French Guiana,50699,181000,28.010497
5,Paramaribo,Suriname,112000,417000,26.858513
6,Lima,Peru,6464693,25662000,25.191696
7,Guayaquil,Ecuador,2070040,12646000,16.369129
8,Santafé de Bogotá,Colombia,6260862,42321000,14.793748
9,Quito,Ecuador,1573458,12646000,12.442337
